<a href="https://colab.research.google.com/github/rbawden/Tutoriel-Normalisation/blob/main/Tutoriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorations sur la normalisation du français moderne

## 1. Setup de l'environnement, téléchargement des fichiers, etc.

Installer les paquets python

In [19]:
!pip install fairseq@git+git://github.com/pytorch/fairseq.git@5a75b079bf8911a327940c28794608e003a9fa52 
!pip install sentencepiece sacrebleu hydra-core omegaconf==2.0.5 gdown==4.2.0 

  Using cached fairseq-1.0.0a0+5a75b07-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached omegaconf-2.0.6-py3-none-any.whl (36 kB)
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 1.4.1
    Uninstalling omegaconf-1.4.1:
      Successfully uninstalled omegaconf-1.4.1


  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.0.6
    Uninstalling omegaconf-2.0.6:
      Successfully uninstalled omegaconf-2.0.6


Télécharger les données et les modèles depuis Google Drive et stocker-les dans `data-models/`

In [3]:
![ -d data-models ] || gdown https://drive.google.com/drive/folders/1h-qSnPBPZFZQ_kqWIBMhkkFS-6C2b10H?usp=sharing -O data-models --folder 

Créer des liens symboliques pour rendre les fichiers/dossiers téléchargés plus accessible

In [40]:
![ -d data ] || mkdir data; [ -d models ] || mkdir models
!for file in data-models/*{src,trg,model,vocab}; do ln -sf  $PWD/$file $PWD/data/; done
!for file in data-models/*{txt,pt}; do ln -sf $PWD/$file $PWD/models/; done
!ln -sf $PWD/models/dict.txt $PWD/models/dict.src.txt
!ln -sf $PWD/models/dict.txt $PWD/models/dict.trg.txt

## 2. Préparation des données à normaliser

Fonctions pour lire le contenu d'un fichier ligne par ligne et pour les lire depuis un fichier

In [42]:
def read_file(filename):
  contents = []
  with open(filename) as fp:
    for line in fp:
      contents.append(line.strip())
  return contents

def write_file(list_sents, filename):
    with open(filename, 'w') as fp:
        for sent in list_sents:
            fp.write(sent + '\n')

Lire le contenu des données parallèles

In [43]:
data_src = read_file('data/dev.src')
data_trg = read_file('data/dev.trg')

Visualiser le début des textes sources (src) et cibles (trg)

In [47]:
for i in range(4):
    print('src = ', data_src[i])
    print('trg = ', data_trg[i])
    print('--')

src =  1.
trg =  1.
--
src =  1. QVe cette propoſtion, qu'vn eſpace eſt vuidé, repugne au ſens commun.
trg =  1. QUe cette proposition, qu'un espace est vidé, répugne au sens commun.
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre du vuide dans leur interualle;
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre du vide dans leur intervalle;
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre ce vide apparent dans leur intervalle:
--


Charger le modèle de segmentation en sous-mots

In [45]:
import sentencepiece
spm = sentencepiece.SentencePieceProcessor(model_file='data/bpe_joint_1000.model')

Appliquer le modèle sur les données

In [46]:
data_src_sp = spm.encode(data_src, out_type=str)
data_trg_sp = spm.encode(data_trg, out_type=str)

Ecrire les données pre-traités dans des fichiers

In [48]:
write_file([' '.join(phrase) for phrase in data_src_sp], 'data/dev.sp.src')
write_file([' '.join(phrase) for phrase in data_trg_sp], 'data/dev.sp.trg')

Visualiser le début de textes

In [12]:
for i in range(4):
    print('src = ', data_src_sp[i])
    print('trg = ', data_trg_sp[i])
    print('--')

src =  ['▁1', '.']
trg =  ['▁1', '.']
--
src =  ['▁1', '.', '▁Q', 'V', 'e', '▁cette', '▁prop', 'ost', 'ion', ',', '▁qu', "'", 'vn', '▁esp', 'ace', '▁est', '▁v', 'ui', 'd', 'é', ',', '▁re', 'p', 'u', 'gne', '▁au', '▁sens', '▁comm', 'un', '.']
trg =  ['▁1', '.', '▁Q', 'U', 'e', '▁cette', '▁prop', 'os', 'it', 'ion', ',', '▁qu', "'", 'un', '▁esp', 'ace', '▁est', '▁v', 'id', 'é', ',', '▁rép', 'u', 'gne', '▁au', '▁sens', '▁comm', 'un', '.']
--
src =  ['▁1', '.', '▁Q', 'V', 'e', '▁tous', '▁les', '▁cor', 'p', 's', '▁ont', '▁re', 'p', 'u', 'gn', 'ance', '▁à', '▁se', '▁se', 'p', 'are', 'r', '▁l', "'", 'vn', '▁de', '▁l', "'", 'autre', ',', '▁&', '▁ad', 'm', 'ettre', '▁du', '▁v', 'ui', 'de', '▁dans', '▁leur', '▁in', 'ter', 'u', 'al', 'le', ';']
trg =  ['▁1', '.', '▁Q', 'U', 'e', '▁tous', '▁les', '▁cor', 'p', 's', '▁ont', '▁rép', 'u', 'gn', 'ance', '▁à', '▁se', '▁s', 'ép', 'are', 'r', '▁l', "'", 'un', '▁de', '▁l', "'", 'autre', ',', '▁et', '▁ad', 'm', 'ettre', '▁du', '▁vi', 'de', '▁dans', '▁leur', 

Définir une fonction pour détokeniser (pour plus tard)

In [51]:
def decode_sp(list_sents):
    return [''.join(sent).replace(' ', '').replace('▁', ' ') for sent in list_sents]

Visualiser à quoi ressemble le texte détokenisé (Spoiler: il devrait ressembler au texte de départ)

In [54]:
decode_sp(data_src_sp[:5])

[' 1.',
 " 1. QVe cette propostion, qu'vn espace est vuidé, repugne au sens commun.",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre du vuide dans leur interualle;",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:",
 ' 2.']

## 3. Appliquer le modèle de normalisation

Appliquer le modèle de normalisation sur le début des données pre-traitées (les messages "User Warning" ne sont pas graves)

In [55]:
!head -n 10 data/dev.sp.src | fairseq-interactive models/ --source-lang src --target-lang trg --path models/model.pt > data/dev.sp.norm.trg.10.output

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/sequence_generator.py:669: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // b

La sortie de fairseq-interactive donne quelque chose comme ceci:

```
S-0     ▁1 .
H-0     -0.00011481383990030736 ▁1 .
P-0     -0.0000 -0.0003 -0.0000
S-1     ▁1 . ▁Q V e ▁cette ▁prop ost ion , ▁qu ' vn ▁esp ace ▁est ▁v ui d é , ▁re p u gne ▁au ▁sens ▁comm un .
H-1     -0.039981111884117126   ▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .
P-1     -0.0000 -0.0000 -0.0043 -0.0632 -0.0006 -0.0000 -0.0001 -0.9353 -0.0001 -0.0012 -0.0000 0.0000 -0.0001 -0.0078 -0.0070 -0.0000 -0.0022 -0.1168 -0.0001 -0.0000 -0.0000 -0.0389 -0.0157 -0.0053 -0.0000 -0.0000 -0.0001 -0.0000 -0.0004 -0.0000
S-2     ▁1 . ▁Q V e ▁tous ▁les ▁cor p s ▁ont ▁re p u gn ance ▁à ▁se ▁se p are r ▁l ' vn ▁de ▁l ' autre , ▁& ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter u al le ;
W-2     0.682   seconds
H-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
D-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
P-2     -0.0000 -0.0001 -0.0040 -0.1684 -0.0004 -0.0000 -0.0000 -0.0000 -0.0007 -0.0000 -0.0001 -0.1220 -0.0063 -0.0002 -0.0137 -0.0000 -0.0000 -0.0002 -0.0001 -0.0248 -0.0022 -0.0003 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0002 -0.0001 -0.0000 -0.0000 -0.0000 -0.0000 -0.0383 -0.0173 -0.0006 -0.0000 -0.0000 -0.0000 -0.0066 -0.0016 -0.4856 -0.0007 -0.0002 -0.0000
```

Les informations intéressantes pour l'exemple `i`:

- S-i: le texte source
- W-i: la durée de la normalisation de ce texte
- H-i: le score de l'hypothèse et l'hypothèse du modèle (c'est-à-dire la prédiction)
- P-i: les scores de chaque sous-token produit par le modèle

Fonction pour extraire l'hypothèse de ce fichier

In [57]:
def extract_hypothesis(filename):
    outputs = []
    with open(filename) as fp:
        for line in fp:
            if 'H-' in line:
                outputs.append(line.strip().split('\t')[2])
    return outputs

Extraire les hypothèses du fichier produit

In [60]:
outputs = extract_hypothesis('data/dev.sp.norm.trg.output')

In [61]:
outputs[:3]

['▁1 .',
 "▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .",
 "▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;"]

Post-traité le texte avec la fonction précedemment définie

In [64]:
outputs_postproc = decode_sp(outputs)

Écrire le résultat dans un fichier

In [65]:
write_file(outputs_postproc, 'data/dev.sp.norm.trg')

## 4. Évaluation du résultat